In [3]:
import pandas as pd 
import numpy as np 
import scipy
# import xlrd 
import sklearn

from Gibbs_model_probit import Gibbs_sampling

from sklearn.model_selection import train_test_split
from scipy.stats import multivariate_normal
from utils import baseline_lr,baseline_esnet,baseline_justmean
from utils import baseline_LogitElsnet,baseline_justmode,baseline_random,baseline_LogitLR,baseline_RanForest
from sklearn.model_selection import KFold
from scipy.stats import binom 
from scipy.stats import norm
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestClassifier
from tqdm import trange
import time

In [4]:
# data_loading (we will use the simu data first)

# data_table = pd.read_csv('../data/processed/all_feature_p2_lip_specie.csv')
data_table = pd.read_csv('../data/processed/all_feature_p2_lip_specie.csv')
# data_table = pd.read_csv('../data/processed/all_feature_p1_lip_specie.csv')
target = 'gap_surv_time_class'
# target = '1= death; 0=alive'



# normalization

# min-max
df = data_table[target]


# check nan
data_table[target].isnull().values.any()
data_table.fillna(data_table.mean(), inplace=True) # fill nan with column mean

In [5]:
gene_feature = data_table.iloc[:,1:775]
# Y = df.values

y = data_table[target].values
print('early dead:%d ,middle:%d,long survive:%d '%((y==0).sum(),(y==2).sum(),(y==1).sum()))
Y = np.where(y>0,1,0)


early dead:48 ,middle:51,long survive:45 


In [6]:
gene_feature

,lip_Sph(d16:1)_27,lip_Sph(d18:1)_27,lip_Sph(d18:2)_27,lip_S1P(d16:1)_6,lip_S1P(d18:0)_6,lip_S1P(d18:1)_6,lip_S1P(d18:2)_6,lip_Cer(d18:0/16:0)_3,lip_Cer(d18:0/18:0)_3,lip_Cer(d18:0/20:0)_3,...,lip_TG(O-54:4) [NL-18:2]_13,lip_Ubiquinone_24,lip_CE(18:2) [+OH]_15,lip_CE(20:4) [+OH]_15,lip_CE(22:6) [+OH]_15,lip_LPC(18:2) [+OH]_15,lip_LPC(20:4) [+OH]_15,lip_LPC(22:6) [+OH]_15,lip_PC(34:2) [+OH]_15,lip_PC(36:4) [+OH]_15
0,1.508517,-1.142981,-0.370896,1.161898,1.393186,2.439157,3.142364,2.807684,1.906719,1.964852,...,0.036007,0.359336,-0.383320,-0.018562,-0.205400,-0.186943,-0.090134,-0.100703,-0.293183,1.595553
1,0.662510,-0.169350,0.276919,-0.420598,0.890299,1.172982,0.270768,1.235381,-0.120123,0.356863,...,-0.656573,-0.557216,4.805999,2.009266,2.187153,11.290922,7.801393,8.273906,10.974755,2.317701
2,0.382097,0.034175,-0.873632,0.263159,0.442293,-0.246913,-0.094627,0.488499,-1.068774,-0.575590,...,2.257551,-0.525753,-0.165085,-0.231418,-0.152552,-0.118845,-0.172070,-0.142195,-0.062192,0.139186
3,-0.505375,-0.856168,-0.309318,-0.876899,0.129396,-0.332582,-0.197593,1.159958,1.085276,0.743728,...,1.085764,0.103507,-0.345465,-0.515169,-0.190832,-0.202784,-0.200526,-0.244971,-0.218103,-1.082805
4,-0.973029,-1.221808,0.084801,1.401033,0.822457,0.869329,0.284968,0.232719,0.030465,-0.985736,...,-1.369647,-1.081999,5.153219,7.136091,8.406937,4.119153,6.956574,8.631769,6.050933,3.055967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,0.836448,0.078052,-0.680276,-0.222538,-1.176785,-1.582970,-1.824283,0.018785,1.688315,0.036113,...,-0.919553,-0.499796,0.454661,0.274775,-0.177827,-0.122108,-0.138097,-0.105071,-0.128006,-0.260284
140,0.840385,0.370737,-0.795050,-0.544087,-0.086341,-0.931198,-1.267266,-0.992808,0.144297,-0.843535,...,-0.504697,-0.009101,0.544398,-0.174427,-0.209744,-0.171430,-0.183107,-0.169494,-0.137458,-0.804552
141,0.958798,-0.450028,0.631433,1.413756,-0.018409,-0.517109,0.988292,-1.232623,0.418946,-0.962435,...,-0.217015,0.367528,0.289700,0.035265,-0.179299,-0.172936,-0.163742,-0.136950,-0.152169,-0.558493
142,-0.486482,-0.715181,-0.765942,-1.725063,-1.572528,-1.536034,-1.350552,-1.357425,-1.695333,-0.907401,...,-0.470357,-0.925047,0.599696,-0.066377,-0.217330,-0.155953,-0.172993,-0.199861,-0.191259,-0.745833


In [7]:
feature_names = list(gene_feature.columns)
K=41 # group number, from data process notebook
group_ind_dict = {}
group_ind = []
group_ind_concat = []
for i in range(K):
    group_ind_dict[i] = []

for name in feature_names:
    id = int(name.split('_')[-1])
    group_ind_dict[id].append(name)

for i in range(K):
    group_ind.append(group_ind_dict[i])
    group_ind_concat = group_ind_concat + group_ind_dict[i]

# group_ind

In [8]:
# re-arrange the features of X based on the group split order
X_new = gene_feature[group_ind_concat].values

N_sample,_ = X_new.shape
# add all-one column at the last 
bias_col = np.ones(N_sample).reshape((N_sample,1))
X_new = np.concatenate((X_new,bias_col),axis=1)

print(X_new.shape)

(144, 775)


In [9]:
# init hyper-parameters
alpha = 0.5
beta = 0.5
r0 = 1e-4
r1 = 1.0
a0 = 1.0
b0 = 1.0
JITTER = 1e-3

INTERVAL = 50
VALITA_INTERVAL = 200
BURNING = 2000
MAX_NUMBER = 5000

hyper_paras = {'INTERVAL':INTERVAL, 'BURNING':BURNING,'MAX_NUMBER':MAX_NUMBER,'VALITA_INTERVAL':VALITA_INTERVAL,
'alpha':alpha, 'beta':beta,'r0':r0,'r1':r1,'JITTER':JITTER}

In [12]:

N = 50
lr_acc = np.zeros(N)
rf_acc = np.zeros(N)
esnet_acc = np.zeros(N)
mode_acc = np.zeros(N)
random_acc = np.zeros(N)

lr_auc = np.zeros(N)
rf_auc = np.zeros(N)
esnet_auc = np.zeros(N)


for i in range(N):
    X_train, X_test, y_train, y_test = train_test_split(X_new, Y.squeeze(),test_size=0.5)
    data_dict = {'X_tr':X_train, 'y_tr':y_train, 'X_test':X_test, 'y_test':y_test}  
    dict_lr = baseline_LogitLR(data_dict)
    dict_els = baseline_LogitElsnet(data_dict)
    dict_rf = baseline_RanForest(data_dict)
    dict_mode = baseline_justmode(data_dict)
    dict_random = baseline_random(data_dict)

    lr_acc[i] = dict_lr['acr']
    esnet_acc[i] = dict_els['acr']
    rf_acc[i] = dict_rf['acr']
    mode_acc[i] = dict_mode['acr']
    random_acc[i] = dict_random['acr']

    lr_auc[i] = dict_lr['auc']
    rf_auc[i] = dict_rf['auc']
    esnet_auc[i] = dict_els['auc']


print('\n\nlr_acr_mean: %.4f,lr_acr_std: %.4f '%(lr_acc.mean(),lr_acc.std() ) )
print('esnet_acr_mean: %.4f,esnet_acr_mean: %.4f '%(esnet_acc.mean(),esnet_acc.std() ) )
print('rf_acr_mean: %.4f,rf_acr_mean: %.4f '%(rf_acc.mean(),rf_acc.std() ) )
print('just-mode_acr_mean: %.4f,mode_acr_std: %.4f '%(mode_acc.mean(),mode_acc.std() ) )
print('just-random_acr_mean: %.4f,just-random_acr_std: %.4f '%(random_acc.mean(),random_acc.std() ) )

print('lr_AUC_mean: %.4f,lr_AUC_std: %.4f '%(lr_auc.mean(),lr_auc.std() ) )
print('esnet_AUC_mean: %.4f,esnet_AUC_mean: %.4f '%(esnet_auc.mean(),esnet_auc.std() ) )
print('rf_AUC_mean: %.4f,rf_AUC_std: %.4f '%(rf_auc.mean(),rf_auc.std() ) )


0.518519
C:\Users\Tianfan Wu\Documents\Python Scripts\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
for LogitLR: accuracy is 0.666667, auc is 0.649091,  fpr is 0.500000
C:\Users\Tianfan Wu\Documents\Python Scripts\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
for LogitElsnet: accuracy is 0.666667, auc is 0.655455,  fpr is 0.500000
C:\Users\Tianfan Wu\Documents\Python Scripts\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
for RandomForest: accuracy is 0.763889, auc is 0.760455,  fpr is 0.681818
for justmode: accuracy is 0.694444, fpr is 1.000000
for random-gue

In [9]:
# # init parameters
# z_array_init = np.random.binomial(size=K, n=1, p= alpha)
# s_list_init = [np.random.binomial(size=len(item), n=1, p= beta) for item in group_ind]
# b_init = np.random.normal(loc=0.0, scale=r1,size=None)
# tau_init = np.random.gamma(shape=alpha, scale=1.0/beta, size=None)
# W_init = []

# for i in range(K):
#     mask1 = 1-z_array_init[i] * s_list_init[i]
#     mask2 = z_array_init[i] * s_list_init[i]
#     spike = np.random.normal(loc=0.0, scale=r0,size=len(s_list_init[i]))
#     slab = np.random.normal(loc=0.0, scale=r1,size=len(s_list_init[i]))
#     W_group = spike * mask1 + slab * mask2

#     W_init.append(W_group)

# init_paras = {'z':z_array_init, 's':s_list_init, 'b':b_init, 'tau':tau_init, 'W':W_init,'a0':a0,'b0':b0}

In [13]:
# init parameters with lr_result
def get_init_paras(w_lr):
    z_array_init = np.ones(K) #np.random.binomial(size=K, n=1, p= alpha)
    s_list_init = [np.random.binomial(size=len(item), n=1, p= beta) for item in group_ind]
    b_init = w_lr[-1]#np.random.normal(loc=0.0, scale=r1,size=None)
    # tau_init = 1.0#np.random.gamma(shape=alpha, scale=1.0/beta, size=None)

    W_init = []
    offset=0
    for i in range(K):
        # mask1 = 1-z_array_init[i] * s_list_init[i]
        # mask2 = z_array_init[i] * s_list_init[i]
        # spike = np.random.normal(loc=0.0, scale=r0,size=len(s_list_init[i]))
        # slab = np.random.normal(loc=0.0, scale=r1,size=len(s_list_init[i]))
        # W_group = spike * mask1 + slab * mask2

        
        group_len = len(s_list_init[i])
        W_group= w_lr[offset:offset+group_len]
        offset = offset + group_len
        W_init.append(W_group)

    init_paras = {'z':z_array_init, 's':s_list_init, 'b':b_init,  'W':W_init,'a0':a0,'b0':b0}
    return init_paras

In [14]:

N = 10
lr_acc = np.zeros(N)
rf_acc = np.zeros(N)
esnet_acc = np.zeros(N)
mode_acc = np.zeros(N)
random_acc = np.zeros(N)
ours_acc = np.zeros(N)


lr_auc = np.zeros(N)
rf_auc = np.zeros(N)
esnet_auc = np.zeros(N)
ours_auc = np.zeros(N)


for i in range(N):
    X_train, X_test, y_train, y_test = train_test_split(X_new, Y.squeeze(),test_size=0.3)

    data_dict = {'X_tr':X_train, 'y_tr':y_train, 'X_test':X_test, 'y_test':y_test}  
    dict_lr = baseline_LogitLR(data_dict)
    dict_els = baseline_LogitElsnet(data_dict)
    dict_rf = baseline_RanForest(data_dict)
    dict_mode = baseline_justmode(data_dict)
    dict_random = baseline_random(data_dict)

    model = Gibbs_sampling(data_dict,get_init_paras(dict_lr['clf'].coef_.squeeze()), hyper_paras)
    dict_ours = model.model_run()

    lr_acc[i] = dict_lr['acr']
    esnet_acc[i] = dict_els['acr']
    rf_acc[i] = dict_rf['acr']
    mode_acc[i] = dict_mode['acr']
    random_acc[i] = dict_random['acr']
    ours_acc[i] = dict_ours['acr']


    lr_auc[i] = dict_lr['auc']
    rf_auc[i] = dict_rf['auc']
    esnet_auc[i] = dict_els['auc']
    ours_auc[i] = dict_ours['auc']


print('\n\nours_acr_mean: %.4f,ours_acr_std: %.4f '%( ours_acc.mean(), ours_acc.std() ) )
print('lr_acr_mean: %.4f,lr_acr_std: %.4f '%(lr_acc.mean(),lr_acc.std() ) )
print('esnet_acr_mean: %.4f,esnet_acr_std: %.4f '%(esnet_acc.mean(),esnet_acc.std() ) )
print('rf_acr_mean: %.4f,rf_acr_mean: %.4f '%(rf_acc.mean(),rf_acc.std() ) )
print('just-mode_acr_mean: %.4f,mode_acr_std: %.4f '%(mode_acc.mean(),mode_acc.std() ) )
print('just-random_acr_mean: %.4f,just-random_acr_std: %.4f '%(random_acc.mean(),random_acc.std() ) )

print('ours_AUC_mean: %.4f,ours_AUC_std: %.4f '%(ours_auc.mean(),ours_auc.std() ) )
print('lr_AUC_mean: %.4f,lr_AUC_std: %.4f '%(lr_auc.mean(),lr_auc.std() ) )
print('esnet_AUC_mean: %.4f,esnet_AUC_mean: %.4f '%(esnet_auc.mean(),esnet_auc.std() ) )
print('rf_AUC_mean: %.4f,rf_AUC_std: %.4f '%(rf_auc.mean(),rf_auc.std() ) )


st-auc = 0.56105
running train-auc = 0.99029

 92%|█████████▏| 4600/5000 [10:14<00:49,  8.03it/s]
 running test-auc = 0.65895
running train-auc = 0.99417

 96%|█████████▌| 4800/5000 [10:40<00:27,  7.19it/s]
 running test-auc = 0.62737
running train-auc = 0.99611

100%|██████████| 5000/5000 [11:07<00:00,  7.49it/s]


 final test auc = 0.54105, acr = 0.52273
C:\Users\Tianfan Wu\Documents\Python Scripts\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
for LogitLR: accuracy is 0.545455, auc is 0.511983,  fpr is 0.588235
C:\Users\Tianfan Wu\Documents\Python Scripts\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
for LogitElsnet: accuracy is 0.545455, auc is 0.570806,  fpr is 0.588235
C:\Users\Tianfan Wu\Documents\Python Scri

In [21]:
print('lr_rmse_mean: %.4f,lr_rmse_std: %.4f '%(lr_rmse[[0,2,3]].mean(),lr_rmse[[0,2,3]].std() ) )
print('esnet_rmse_mean: %.4f,esnet_rmse_std: %.4f '%(esnet_rmse[[0,2,3]].mean(),esnet_rmse[[0,2,3]].std() ) )
print('ours_rmse_mean: %.4f,ours_rmse_std: %.4f '%(ours_rmse[[0,2,3]].mean(),ours_rmse[[0,2,3]].std() ) )

lr_rmse_mean: 1.3432,lr_rmse_std: 0.1101 
esnet_rmse_mean: 1.1714,esnet_rmse_std: 0.2027 
ours_rmse_mean: 1.1854,ours_rmse_std: 0.1600 


(array([ 0, 24, 26], dtype=int64),)

In [15]:
model.z_mean

array([1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1])

In [86]:
from sklearn.linear_model import ElasticNet

clf = ElasticNet(alpha = 0.2,l1_ratio =0.1, fit_intercept=False)
clf.fit(data_dict['X_tr'], data_dict['y_tr'])
predict = clf.predict(data_dict['X_test']).squeeze()
rmse = np.sqrt(np.mean((predict-data_dict['y_test'].squeeze())**2))
print('with elanet, rmse is %.5f'%(rmse))
len(clf.coef_[clf.coef_>0])

with elanet, rmse is 0.92433


114

In [14]:
w_es

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.02290048, -0.00710251, -0.        ,
       -0.        , -0.00906848, -0.        , -0.00073689, -0.        ,
       -0.01301175,  0.        ,  0.        , -0.        ,  0.04461527,
       -0.        ,  0.        , -0.        , -0.        , -0.        ,
       -0.        ,  0.        ,  0.        , -0.        ,  0.04582258,
       -0.        , -0.        ,  0.        , -0.00123495, -0.00794974,
       -0.01668715, -0.        , -0.        ,  0.        ,  0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.05419244,  0.        , -0.        , -0.05123613,  0.00319208,
       -0.11459053,  0.05525388,  0.        ,  0.02155437,  0.        ,
        0.02790468,  0.        , -0.        , -0.        ,  0.        ,
       -0.        , -0.        ,  0.        ,  0.        ,  0.  